In [1]:
import boto3

bucket_list = []

s3 = boto3.resource('s3')
s3_object = s3.Bucket('citibikejsonfiles')

for file in s3_object.objects.filter(Prefix = 'dataStationInformation'):
    file_name = file.key
    if file_name.find(".json") != -1:
        bucket_list.append(file.key)

len_bucket_list = print(len(bucket_list))
print(bucket_list[0:10])


857
['dataStationInformation 2023-01-14 01:40:37.364932.json', 'dataStationInformation 2023-01-14 01:50:37.351757.json', 'dataStationInformation 2023-01-14 02:00:37.354085.json', 'dataStationInformation 2023-01-14 02:10:37.297757.json', 'dataStationInformation 2023-01-14 02:20:37.491889.json', 'dataStationInformation 2023-01-14 02:30:37.376058.json', 'dataStationInformation 2023-01-14 02:40:37.058998.json', 'dataStationInformation 2023-01-14 02:50:37.666289.json', 'dataStationInformation 2023-01-14 03:00:37.288828.json', 'dataStationInformation 2023-01-14 03:10:37.558505.json']


In [2]:
dataframe_key = []

for file in bucket_list:
    file_name = file[23:39]
    dataframe_key.append(file_name)

dataframe_key[:5]

['2023-01-14 01:40',
 '2023-01-14 01:50',
 '2023-01-14 02:00',
 '2023-01-14 02:10',
 '2023-01-14 02:20']

In [3]:
from io import StringIO
import io
import pandas as pd

dataframe = [] # empty list of dataframes

for file in bucket_list:
    s3_obj = s3.Object('citibikejsonfiles', file)
    data = s3_obj.get()['Body'].read()
    dataframe.append(pd.read_json(io.BytesIO(data)))

In [4]:
appended_data = []
for key in range(len(dataframe)):
    tmpDataStationInformation = pd.json_normalize(dataframe[key]['data']['stations'])
    tmpDataStationInformation['time_stamp'] = dataframe_key[key]
    appended_data.append(tmpDataStationInformation)
StationInformationTimeSeries = pd.concat(appended_data, ignore_index = True)

In [5]:
StationInformationTimeSeries.head()

,eightd_has_key_dispenser,short_name,capacity,rental_methods,name,lon,eightd_station_services,station_type,electric_bike_surcharge_waiver,has_kiosk,lat,legacy_id,external_id,region_id,station_id,rental_uris.ios,rental_uris.android,time_stamp
0,False,JC003,22,"[KEY, CREDITCARD]",City Hall,-74.043845,[],classic,False,True,40.717732,3185,66dd5960-0aca-11e7-82f6-3863bb44ef7c,70,66dd5960-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
1,False,JC002,14,"[KEY, CREDITCARD]",Paulus Hook,-74.033552,[],classic,False,True,40.714145,3184,66dd58f2-0aca-11e7-82f6-3863bb44ef7c,70,66dd58f2-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
2,False,JC008,14,"[KEY, CREDITCARD]",Newport Pkwy,-74.032108,[],classic,False,True,40.728745,3199,66dd6441-0aca-11e7-82f6-3863bb44ef7c,70,66dd6441-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
3,False,JC009,26,"[KEY, CREDITCARD]",Hamilton Park,-74.044247,[],classic,False,True,40.727596,3203,66dd7b37-0aca-11e7-82f6-3863bb44ef7c,70,66dd7b37-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40
4,False,JC006,22,"[KEY, CREDITCARD]",Warren St,-74.038051,[],classic,False,True,40.721124,3187,66dd5a42-0aca-11e7-82f6-3863bb44ef7c,70,66dd5a42-0aca-11e7-82f6-3863bb44ef7c,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan,2023-01-14 01:40


In [23]:
StationInformationTimeSeries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1835 entries, 0 to 1834
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   eightd_station_services         1835 non-null   object 
 1   station_id                      1835 non-null   object 
 2   station_type                    1835 non-null   object 
 3   has_kiosk                       1835 non-null   bool   
 4   eightd_has_key_dispenser        1835 non-null   bool   
 5   legacy_id                       1835 non-null   object 
 6   region_id                       1834 non-null   object 
 7   external_id                     1835 non-null   object 
 8   capacity                        1835 non-null   int64  
 9   short_name                      1835 non-null   object 
 10  name                            1835 non-null   object 
 11  lat                             1835 non-null   float64
 12  rental_methods                  18